In [1]:
import findspark
import pandas as pd
findspark.init()

from pyspark.sql import SparkSession
from pyspark import SparkConf

# for shared metastore (shared across all users)
spark = SparkSession.builder.appName("Testing").config("hive.metastore.uris", "thrift://bialobog:9083", conf=SparkConf()).getOrCreate() \

# for local metastore (your private, invidivual database) add the following config to spark session
spark.sql("USE 2023_04_01")

DataFrame[]

In [2]:
import pyspark.pandas as ps
from pyspark.sql.functions import lit,col
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import xgboost as xgb
#from boruta import BorutaPy
#from fredapi import Fred
from sklearn.linear_model import Lasso
from sklearn.model_selection import TimeSeriesSplit
import csv
from pyspark.sql import functions as F
from functools import reduce
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf
from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, CrossValidatorModel
from CreateDataset import get_features_all_stocks_seq, get_full_series_stocks, get_tabular_dataset
from pyspark.sql.window import Window

/opt/spark/python/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [3]:
df = get_tabular_dataset()

In [ ]:
def split_train_test(df, split_date):
    split_date = pd.to_datetime(split_date)
    train_df = df.filter(col('date')<split_date)
    test_df = df.filter(col('date')>=split_date)
    return train_df, test_df


def forward_fill(df):
    window_spec = Window.partitionBy('fsym_id').orderBy('date')
    feature_cols = df.columns[2:-1]
    for c in feature_cols:
        df = df.withColumn(
            c, F.last(c, ignorenulls=True).over(window_spec)
        )
    return df.orderBy('fsym_id','date')

def scale():
    pass

In [6]:
from Boruta import BorutaPy
import xgboost as xgb


def boruta_fs(df): #HOW DOES BORUTA ACC WORK?
    train_df, test_df = split_train_test(df, '2019-12-31')
    train_df = train_df.toPandas()
    test_df = test_df.toPandas()
    X_train = train_df.drop(['fsym_id', 'date', 'label'], axis=1)
    y_train = train_df['label']
    X_test = test_df.drop(['fsym_id', 'date', 'label'], axis=1)
    y_test = test_df['label']
    
    model = xgb.XGBClassifier()
    feat_selector = BorutaPy(model, n_estimators='auto', verbose=2, random_state=1)
    feat_selector.fit(X_train, y_train)
    
    print(feat_selector.support_)
    print(feat_selector.ranking_)
    
    
boruta_fs(df)
    

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	14
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	14
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	14
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	14
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	14
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	14
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	14
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	3
Tentative: 	7
Rejected: 	4
Iteration: 	9 / 100
Confirmed: 	3
Tentative: 	7
Rejected: 	4
Iteration: 	10 / 100
Confirmed: 	3
Tentative: 	7
Rejected: 	4
Iteration: 	11 / 100
Confirmed: 	3
Tentative: 	7
Rejected: 	4
Iteration: 	12 / 100
Confirmed: 	3
Tentative: 	6
Rejected: 	5
Iteration: 	13 / 100
Confirmed: 	3
Tentative: 	6
Rejected: 	5
Iteration: 	14 / 100
Confirmed: 	3
Tentative: 	6
Rejected: 	5
Iteration: 	15 / 100
Confirmed: 	3
Tentative: 	6
Rejected: 	5
Iteration: 	16 / 100
Confirmed: 	3
Tentative: 	6
Rejected: 	5
Iteration:

In [4]:
#df.show()
feature_cols = df.columns[2:-1]
window_spec = Window.partitionBy('fsym_id').orderBy('date')
df.orderBy('fsym_id','date').show()

#FORWARD FILLING NULLS
# for c in feature_cols:
#     df = df.withColumn(
#         c, F.last(c, ignorenulls=True).over(window_spec)
#     )


+--------+----------+-----------------+-----------+-------------------------+--------------------------+--------------------------+--------+-----------------------+----------------------+----------+-------------------+--------------+-----------------------+-----------------+----------------------+-----+
| fsym_id|      date|     ff_assets_gr|ff_earn_yld|ff_net_inc_basic_aft_xord|ff_net_inc_dil_bef_unusual|ff_net_inc_basic_beft_xord|ff_wkcap|ff_oper_inc_aft_unusual|ff_net_inc_bef_xord_gr|ff_fcf_yld|    ff_non_oper_exp|ff_net_inc_dil|ff_net_inc_dil_aft_xord|   ff_oper_inc_gr|ff_ut_non_oper_inc_oth|label|
+--------+----------+-----------------+-----------+-------------------------+--------------------------+--------------------------+--------+-----------------------+----------------------+----------+-------------------+--------------+-----------------------+-----------------+----------------------+-----+
|B00FG1-R|2012-12-31|             null|       null|                    -0.59|        

In [5]:


def model_testing(train_df, test_df):
    feature_cols = df.columns[2:-1]
    assembler = VectorAssembler(inputCols=feature_cols, outputCol='features')
    
    classifiers = [
        LogisticRegression,
        RandomForestClassifier(),
        GBTClassifier()
    ]
    
    cv_models = []
    
    evaluator = BinaryClassificationEvaluator(metricName='areaUnderROC') #SHOULD USE TIMESERIES SPLIT
    
    for classifier in classifiers:
        pipeline = Pipeline(stages=[assembler, classifier])
        
        param_grid = ParamGridBuilder().build()
        
        cross_val = CrossValidator(
            estimator=pipeline,
            estimatorParamMaps=param_grid,
            evaluator=evaluator,
            numFolds=5
        )
        
        cv_model = cross_val.fit(train_df)
        cv_models.append(cv_model)
        print("Done")
    
    for cv_model in cv_models:
        predictions = cv_model.transform(test_df)
        auc = evaluator.evaluate(predictions)
        print(f"Model AUC: {auc}")

df = forward_fill(df)
print(df.count())
print(df.filter(F.col('label')==1).count())
df=df.na.drop()
print(df.count())
print(df.filter(F.col('label')==1).count())

train_df, test_df = split_train_test(df, '2019-12-31')
print(train_df.count()/df.count())
print(test_df.count()/df.count())
print(test_df.filter(col('label')==1).count())

model_testing(train_df, test_df)

114217
553
76565
477
0.7896297263762816
0.2103702736237184
74
Model AUC: 0.8931001262598599
Model AUC: 0.8984509988688874
